# Azure Storage에서 엑셀 연결하여 uri 가져오기

필요한 라이브러리 설치

In [ ]:
# pip install azure-storage-blob pandas openpyxl

In [1]:
import pandas as pd
import os
from azure.storage.blob import BlobServiceClient

Azure Storage 정보

In [ ]:
# 1️⃣ Azure Storage 설정
STORAGE_CONNECTION_STRING = ""
CONTAINER_NAME = "clothimage"
STORAGE_ACCOUNT_NAME = "chilldata"

# 2️⃣ 엑셀 파일 경로
EXCEL_PATH = "outer_data.xlsx"

### outer data로 미리 해보기

In [8]:
# 3️⃣ Azure Blob URL 생성 함수
def generate_blob_url(file_name):
    return f"https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/{CONTAINER_NAME}/{file_name}"

# 4️⃣ 엑셀 데이터 업데이트
def update_excel_with_blob_urls(excel_path):
    df = pd.read_excel(excel_path)

    # 🔥 컬럼명 정리 (공백 제거)
    df.columns = df.columns.str.strip()
    print("📌 엑셀 컬럼명:", df.columns)  # 컬럼 확인

    # ✅ "imageFileName" 열이 있는지 확인 후 처리
    if "imageFileName" not in df.columns:
        raise KeyError("❌ 'imageFileName' 열이 엑셀에 없습니다. 열 이름을 확인하세요.")

    for index, row in df.iterrows():
        file_name = str(row["imageFileName"]).strip()  # 파일명 정리
        if pd.notna(file_name):  # 파일명이 있는 경우
            df.at[index, "imageLinkBlob"] = generate_blob_url(file_name)

    # 5️⃣ 엑셀 업데이트 (덮어쓰기)
    df.to_excel(excel_path, index=False)
    print(f"✅ {excel_path} 업데이트 완료!")

# 실행
update_excel_with_blob_urls(EXCEL_PATH)


📌 엑셀 컬럼명: Index(['ID', 'brand', 'itemName', 'price', 'productLink', 'imageFileName',
       'imageLinkBlob'],
      dtype='object')
✅ outer_data.xlsx 업데이트 완료!


/var/folders/n_/2h1xg2xs7dxdc2fc9ywry06w0000gn/T/ipykernel_3588/1758479277.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'https://chilldata.blob.core.windows.net/clothimage/O1.jpg' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, "imageLinkBlob"] = generate_blob_url(file_name)


# 엑셀 전처리
  - 전체 데이터

In [ ]:
import pandas as pd
import re

def update_ids(file_path, save_path):
    df = pd.read_excel(file_path)
    
    outer_count = 1
    top_count = 1
    pants_count = 1
    dress_skirt_count = 1
    
    def generate_id(image_name):
        nonlocal outer_count, top_count, pants_count, dress_skirt_count
        
        if re.match(r"O\d+\.jpg", image_name):
            new_id = f"outer_id_{outer_count}"
            outer_count += 1
        elif re.match(r"T\d+\.jpg", image_name):
            new_id = f"top_id_{top_count}"
            top_count += 1
        elif "pants" in image_name.lower() or re.match(r"Dress_skirt_\d+\.jpg", image_name):
            new_id = f"pants_id_{pants_count}"
            pants_count += 1
        elif "dress_skirt" in image_name.lower():
            new_id = f"dress_skirt_id_{dress_skirt_count}"
            dress_skirt_count += 1
        else:
            new_id = "unknown_id"
        
        return new_id
    
    df["ID"] = df["imageFileName"].apply(generate_id)
    df.to_excel(save_path, index=False)
    print(f"Updated file saved to: {save_path}")

# 파일 경로 설정
file_path = "./data/clothimage.xlsx"
save_path = "./data/updated_clothimage.xlsx"
update_ids(file_path, save_path)


Updated file saved to: updated_clothimage.xlsx


# 전체 데이터 엑셀로 Azure URI도 정리

In [ ]:
# 1️⃣ Azure Storage 설정


In [12]:
# 3️⃣ Azure Blob URL 생성 함수
def generate_blob_url(file_name):
    return f"https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/{CONTAINER_NAME}/{file_name}"

# 4️⃣ 엑셀 데이터 업데이트
def update_excel_with_blob_urls(excel_path):
    df = pd.read_excel(excel_path)

    # 🔥 컬럼명 정리 (공백 제거)
    df.columns = df.columns.str.strip()
    print("📌 엑셀 컬럼명:", df.columns)  # 컬럼 확인

    # ✅ "imageFileName" 열이 있는지 확인 후 처리
    if "imageFileName" not in df.columns:
        raise KeyError("❌ 'imageFileName' 열이 엑셀에 없습니다. 열 이름을 확인하세요.")

    for index, row in df.iterrows():
        file_name = str(row["imageFileName"]).strip()  # 파일명 정리
        if pd.notna(file_name):  # 파일명이 있는 경우
            df.at[index, "imageLinkBlob"] = generate_blob_url(file_name)

    # 5️⃣ 엑셀 업데이트 (덮어쓰기)
    df.to_excel(excel_path, index=False)
    print(f"✅ {excel_path} 업데이트 완료!")

# 실행
update_excel_with_blob_urls(EXCEL_PATH)

📌 엑셀 컬럼명: Index(['ID', 'brand', 'itemName', 'price', 'productLink', 'imageFileName',
       'imageLinkBlob'],
      dtype='object')


/var/folders/n_/2h1xg2xs7dxdc2fc9ywry06w0000gn/T/ipykernel_3588/2020684113.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'https://chilldata.blob.core.windows.net/clothimage/O1.jpg' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, "imageLinkBlob"] = generate_blob_url(file_name)


✅ data/updated_clothimage.xlsx 업데이트 완료!
